/bin/sh: 1: pip: not found


In [1]:
# The essentials
import pandas as pd
import numpy as np

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Progress bars
from tqdm import tqdm

# Gradient Boosting
from catboost import CatBoostRegressor

from collections import defaultdict

In [2]:
%pwd

'/home/kiani/mlcomp/Rinse-Over-Run'

In [3]:

train_df = pd.read_csv('./data/train_values.csv', index_col=0, parse_dates=['timestamp'])
test_df = pd.read_csv('./data/test_values.csv', index_col=0, parse_dates=['timestamp'])
label_df = pd.read_csv('./data/train_labels.csv', index_col='process_id')

/home/kiani/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
# variables we'll use to create our time series features
ts_cols = [
    'process_id',
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
]

# variables for binary time series features
bin_cols = [
    'supply_pump',
    'supply_pre_rinse',
    'supply_caustic',
    'return_caustic',
    'supply_acid',
    'return_acid',
    'supply_clean_water',
    'return_recovery_water',
    'return_drain',
    'object_low_level'
]

# phases, ordered from earliest to latest
phases = ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid']

In [5]:
def encode_categorical(df):
    # Currently just copy-pasted from http://drivendata.co/blog/rinse-over-run-benchmark/
    
    # select process_id and pipeline
    meta = df[['process_id', 'pipeline']].drop_duplicates().set_index('process_id') 
    
    # convert categorical pipeline data to dummy variables
    meta = pd.get_dummies(meta)
    
    # pipeline L12 not in test data (so useless feature)
    if 'pipeline_L12' in meta:
        meta = meta.drop('pipeline_L12', axis=1)
    
    # calculate number of phases for each process_object
    meta['num_phases'] = df.groupby('process_id')['phase'].apply(lambda x: x.nunique())
    
    return meta

def percentile_25(x):
  return np.percentile(x, 0.25)

def percentile_75(x):
  return np.percentile(x, 0.75)
  
def encode_real_timeseries(df):   
    ts_df = df[ts_cols].set_index('process_id')
    
    # create features: count, min, max, mean, standard deviation
    ts_features = ts_df.groupby('process_id').agg(['min', 'max', 'mean', 'std', 
                                                   'count', 'median', 'sum', 
                                                   'mad', percentile_25,
                                                   percentile_75])
    
    # Now we will get the mean + variance value of the last K measurements for each phase
    all_vals_per_phase = []
    K = 5
    col_names = ['process_id'] 
    for phase in phases:
        for col in ts_cols:
            col_names.extend(['mean_{}_{}_{}'.format(col, K, phase), 
                              'std_{}_{}_{}'.format(col, K, phase)])
    for process in tqdm(ts_features.index, total=len(ts_features)):
        vals_per_phase = [process]
        process_filtered_df = df[df['process_id'] == process]
        for phase in phases:
            filtered_df = process_filtered_df[process_filtered_df['phase'] == phase].tail(K)
            for col in ts_cols:
                vals_per_phase.extend([filtered_df[col].mean(), filtered_df[col].std()])
                
        all_vals_per_phase.append(vals_per_phase)
    values_df = pd.DataFrame(all_vals_per_phase, columns=col_names)
    values_df = values_df.set_index('process_id')
    
    ts_features = ts_features.merge(values_df, left_index=True, right_index=True)
    
    return ts_features

def encode_binary_timeseries(df):
    ts_df = df[ts_cols].set_index('process_id')
            
    # create features: count, min, max, mean, standard deviation
    ts_features = ts_df.groupby('process_id').agg(['mean', 'std', 'count', 'sum', 'mad'])
    
    # TODO: Count fraction of True in each phase
    feature_vectors = []
    col_names = ['process_id'] 
    for phase in phases:
        for col in bin_cols:
            col_names.append('fraction_{}_{}'.format(col, phase))
            
    # Get fraction of True values for each binary timeseries
    for process in tqdm(set(df['process_id']), total=len(set(df['process_id']))):
        vector = [process]
        process_filtered_df = df[df['process_id'] == process]
        for phase in phases:
            filtered_df = process_filtered_df[process_filtered_df['phase'] == phase]
            for col in bin_cols:
                if len(filtered_df):
                    vector.append(sum(filtered_df[col]) / len(filtered_df))
                else:
                    vector.append(np.NaN)
                
        feature_vectors.append(vector)
                
    feature_df = pd.DataFrame(feature_vectors, columns=col_names)
    feature_df = feature_df.set_index('process_id')
    
    feature_df = feature_df.merge(ts_features, left_index=True, right_index=True)
    
    return feature_df

def create_feature_matrix(df):
    metadata = encode_categorical(df)
    time_series = encode_real_timeseries(df)
    binary_features = encode_binary_timeseries(df)
    
    # join metadata and time series features into a single dataframe
    feature_matrix = metadata
    feature_matrix = feature_matrix.merge(time_series, left_index=True, right_index=True)
    feature_matrix = feature_matrix.merge(binary_features, left_index=True, right_index=True)
    
    return feature_matrix

In [8]:
process_comb = 15
test_features_index = pd.read_csv('./data/test_features_{}.csv'.format(process_comb), index_col=['process_id']).index
train_features_index = pd.read_csv('./data/train_features_adv_{}.csv'.format(process_comb), index_col=['process_id']).index
val_features_index = pd.read_csv('./data/val_features_adv_{}.csv'.format(process_comb), index_col=['process_id']).index

In [11]:
train_df_no_final = train_df[train_df['phase'] != 'final_rinse']

X_train = create_feature_matrix(train_df_no_final[train_df_no_final['process_id'].isin(train_features_index)])
X_val = create_feature_matrix(train_df_no_final[train_df_no_final['process_id'].isin(val_features_index)])
X_test = create_feature_matrix(test_df[test_df['process_id'].isin(test_features_index)])

  0%|          | 0/3354 [00:00<?, ?it/s]

  0%|          | 7/3354 [00:00<00:53, 62.35it/s]

  0%|          | 13/3354 [00:00<00:56, 59.17it/s]

  1%|          | 19/3354 [00:00<00:57, 58.47it/s]

  1%|          | 25/3354 [00:00<00:56, 58.46it/s]

...

 92%|█████████▏| 617/671 [00:04<00:00, 149.53it/s]

 94%|█████████▍| 632/671 [00:04<00:00, 147.01it/s]

 97%|█████████▋| 648/671 [00:04<00:00, 149.14it/s]

 99%|█████████▉| 664/671 [00:04<00:00, 150.93it/s]

100%|██████████| 671/671 [00:04<00:00, 150.58it/s]

<bound method NDFrame.head of          process_id  object_id        phase           timestamp pipeline  \
row_id                                                                     
0             20001        405    pre_rinse 2018-04-15 04:20:47       L4   
1             20001        405    pre_rinse 2018-04-15 04:20:49       L4   
2             20001        405    pre_rinse 2018-04-15 04:20:51       L4   
3             20001        405    pre_rinse 2018-04-15 04:20:53       L4   
4             20001        405    pre_rinse 2018-04-15 04:20:55       L4   
5             20001        405    pre_rinse 2018-04-15 04:20:57       L4   
6             20001        405    pre_rinse 2018-04-15 04:20:59       L4   
7             20001        405    pre_rinse 2018-04-15 04:21:01       L4   
8             20001        405    pre_rinse 2018-04-15 04:21:03       L4   
9             20001        405    pre_rinse 2018-04-15 04:21:05       L4   
10            20001        405    pre_rinse 2018-04-15 04:

In [ ]:


# Getting the duration of the processes
train_df_final = train_df[train_df['phase'] == 'final_rinse']



durations = []
for ix in X_train.index:
    kiesmaar = train_df_final[train_df_final['process_id'] == ix]['target_time_period'].astype(float).values
    durations.append(sum(kiesmaar == 1)/len(kiesmaar))
y_train = pd.Series(data=durations, index=X_train.index)

durations = []
for ix in X_val.index:
    kiesmaar = train_df_final[train_df_final['process_id'] == ix]['target_time_period'].astype(float).values
    durations.append(sum(kiesmaar == 1)/len(kiesmaar))
y_val = pd.Series(data=durations, index=X_val.index)


<bound method NDFrame.head of          process_id  object_id        phase           timestamp pipeline  \
row_id                                                                     
0             20001        405    pre_rinse 2018-04-15 04:20:47       L4   
1             20001        405    pre_rinse 2018-04-15 04:20:49       L4   
2             20001        405    pre_rinse 2018-04-15 04:20:51       L4   
3             20001        405    pre_rinse 2018-04-15 04:20:53       L4   
4             20001        405    pre_rinse 2018-04-15 04:20:55       L4   
5             20001        405    pre_rinse 2018-04-15 04:20:57       L4   
6             20001        405    pre_rinse 2018-04-15 04:20:59       L4   
7             20001        405    pre_rinse 2018-04-15 04:21:01       L4   
8             20001        405    pre_rinse 2018-04-15 04:21:03       L4   
9             20001        405    pre_rinse 2018-04-15 04:21:05       L4   
10            20001        405    pre_rinse 2018-04-15 04:

In [30]:
def custom_mape(approxes, targets):
    return np.mean(np.abs(np.subtract(approxes, targets)) / np.maximum(np.abs(targets), 290000))

class MAPEMetric(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, targets, weight):
        return custom_mape(np.exp(approxes), np.exp(targets)), len(targets)

In [48]:
from sklearn.model_selection import KFold
def mae(real, preds):
  return np.mean(np.abs((real - preds)))

FOLDS = 5
train_predictions = np.zeros((len(train_features_index), 1))
val_predictions = np.zeros((len(val_features_index), FOLDS))
test_predictions = np.zeros((len(test_features_index), FOLDS))

kf = KFold(n_splits=FOLDS)
for fold_nr, (train_index, test_index) in enumerate(kf.split(train_features_index, y_train)):
    X_cv_train = X_train.iloc[train_index, :]
    X_cv_test = X_train.iloc[test_index, :]
    y_cv_train = y_train.iloc[train_index]
    y_cv_test = y_train.iloc[test_index]
    cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100,
                        loss_function='MAE', eval_metric='MAE')
    cat.fit(X_cv_train, y_cv_train, eval_set=(X_val, y_val), verbose=50)
    
    # predictions for out of sample
    pred = cat.predict(X_cv_test)
    train_predictions[test_index] = np.reshape(pred, (-1,1))
    print(mae(y_cv_test, pred))
        
    pred = cat.predict(X_val)
    val_predictions[:, fold_nr] = pred
    print(mae(y_val, pred))
        
    pred = cat.predict(X_test)
    test_predictions[:, fold_nr] = pred

0:	learn: 0.5208251	test: 0.4995055	best: 0.4995055 (0)	total: 201ms	remaining: 5h 34m 41s


50:	learn: 0.0464955	test: 0.0540162	best: 0.0540162 (50)	total: 9.15s	remaining: 4h 58m 57s


100:	learn: 0.0259222	test: 0.0370071	best: 0.0369470 (98)	total: 17.2s	remaining: 4h 43m 1s


150:	learn: 0.0203993	test: 0.0346711	best: 0.0345166 (143)	total: 25.3s	remaining: 4h 39m 10s


200:	learn: 0.0170347	test: 0.0353841	best: 0.0342180 (165)	total: 33.4s	remaining: 4h 36m 43s


...

50:	learn: 0.0469387	test: 0.0545256	best: 0.0545256 (50)	total: 8.37s	remaining: 4h 33m 18s


100:	learn: 0.0251351	test: 0.0362646	best: 0.0362646 (100)	total: 16.5s	remaining: 4h 32m 28s


150:	learn: 0.0209689	test: 0.0352399	best: 0.0351775 (149)	total: 27.6s	remaining: 5h 4m 25s


200:	learn: 0.0181451	test: 0.0364376	best: 0.0351775 (149)	total: 36.3s	remaining: 5h 34s


Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.03517753338
bestIteration = 149

Shrink model to first 150 iterations.
0.02851923073983253
0.03517753200823083


In [51]:
y_train = np.log(label_df.loc[X_train.index])
y_val = np.log(label_df.loc[X_val.index])



In [ ]:

cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100,
                        loss_function='MAPE', eval_metric=MAPEMetric(), learning_rate=0.33)
cat.fit(X_train, y_train, eval_set=(X_val,y_val), verbose=50)


In [52]:
train_duration = pd.DataFrame(train_predictions, columns=["duration_pred"], index=X_train.index)
val_duration = pd.DataFrame(np.mean(val_predictions, axis=1), columns=["duration_pred"], index=X_val.index)
X_train_aug = X_train.merge(train_duration, left_index=True, right_index=True)
X_val_aug = X_val.merge(val_duration, left_index=True, right_index=True)

cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100,
                        loss_function='MAPE', eval_metric=MAPEMetric(), learning_rate=0.33)
cat.fit(X_train_aug, y_train, eval_set=(X_val_aug,y_val), verbose=50)

0:	learn: 0.8490310	test: 0.8194781	best: 0.8194781 (0)	total: 209ms	remaining: 5h 48m 32s


50:	learn: 0.8490263	test: 0.8194731	best: 0.8194731 (50)	total: 7.95s	remaining: 4h 19m 37s


100:	learn: 0.8490085	test: 0.8194538	best: 0.8194538 (100)	total: 10.4s	remaining: 2h 51m 13s


150:	learn: 0.8489416	test: 0.8193793	best: 0.8193793 (150)	total: 12.7s	remaining: 2h 20m 1s


200:	learn: 0.8486870	test: 0.8190894	best: 0.8190894 (200)	total: 15.2s	remaining: 2h 5m 33s


...

800:	learn: 0.2121644	test: 0.2682910	best: 0.2674452 (753)	total: 1m 44s	remaining: 3h 35m 42s


850:	learn: 0.2053466	test: 0.2659511	best: 0.2658358 (848)	total: 1m 52s	remaining: 3h 39m 4s


900:	learn: 0.1989148	test: 0.2667936	best: 0.2657464 (857)	total: 2m 1s	remaining: 3h 41m 55s


950:	learn: 0.1918405	test: 0.2663390	best: 0.2657464 (857)	total: 2m 9s	remaining: 3h 44m 40s


Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2657464138
bestIteration = 857

Shrink model to first 858 iterations.


0:	learn: 298.5597703	test: 312.7481605	best: 312.7481605 (0)	total: 259ms	remaining: 7h 11m 24s
50:	learn: 71.0951036	test: 75.5327746	best: 75.5327746 (50)	total: 20.6s	remaining: 11h 14m 5s
100:	learn: 37.4658724	test: 47.0245748	best: 47.0245748 (100)	total: 41.8s	remaining: 11h 29m 5s
150:	learn: 33.1584354	test: 43.5585793	best: 43.5585793 (150)	total: 1m 2s	remaining: 11h 33m 3s
200:	learn: 31.4163617	test: 41.7790620	best: 41.7790620 (200)	total: 1m 23s	remaining: 11h 31m 25s
250:	learn: 30.1679374	test: 40.6092440	best: 40.5735845 (248)	total: 1m 44s	remaining: 11h 30m 1s
300:	learn: 29.0022957	test: 40.2053943	best: 40.0969311 (294)	total: 2m 4s	remaining: 11h 24m 46s
350:	learn: 27.4965189	test: 39.4655817	best: 39.4143498 (347)	total: 2m 24s	remaining: 11h 24m 32s
400:	learn: 26.4096227	test: 39.0341938	best: 39.0223629 (399)	total: 2m 45s	remaining: 11h 24m 42s
450:	learn: 25.4555063	test: 38.9047251	best: 38.7036650 (441)	total: 3m 5s	remaining: 11h 24m
500:	learn: 24.592

In [22]:
def mae(real, preds):
  return np.mean(real - preds)

mae(cat.predict(X_val), y_val)

7.393971068141912

In [24]:
min(y_val)

142

In [0]:
# Getting the duration of the processes
train_df_final = train_df[train_df['phase'] == 'final_rinse']

durations = []
for ix in X_train.index:
  durations.append(sum(train_df_final[train_df_final['process_id'] == ix]['target_time_period'].astype(float).values == 0))
y_train = pd.Series(data=durations, index=X_train.index)

durations = []
for ix in X_val.index:
  durations.append(sum(train_df_final[train_df_final['process_id'] == ix]['target_time_period'].astype(float).values == 0))
y_val = pd.Series(data=durations, index=X_val.index)

In [0]:
cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100,
                        loss_function='RMSE', eval_metric='RMSE')
cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=50)

0:	learn: 174.7898559	test: 196.3681760	best: 196.3681760 (0)	total: 442ms	remaining: 12h 16m 14s
